In [ ]:
import math
from pint import UnitRegistry
from tabulate import tabulate


units = UnitRegistry()

G = 9.81 * units.meter / units.second**2
EFFICIENCY = 0.8

#   Common CRF (Coefficient of Rolling Friction) values
#     0.02		car tires on tar or asphalt
#     0.06		car tire on solid sand, gravel loose worn, soil medium hard
#     0.35      car tire on field
#     0.40		car tire on loose sand
FRICTION_COEFF = 0.1


def calculate_motor_requirements(
    robot_mass,
    robot_num_powered_wheels,
    robot_wheel_radius,
    speed,
    accel,
    incline_grade,
):
    # Force is needed to overcome friction and accelerate the robot
    #   Force = Force_accel          +  Force_friction
    #         = m * (a + g sin(th))  +  crf * m * g * cos(th)
    #   Units: kg * m / s2

    force_acceleration = robot_mass * (accel + G * math.sin(incline_grade))
    force_acceleration.ito("N")
    force_friction = FRICTION_COEFF * robot_mass * G * math.cos(incline_grade)
    force_friction.ito("N")

    # Torque is the force needed to rotate an object
    #   Torue = Force * Radius
    #   Units: N m

    torque_for_accel = force_acceleration * robot_wheel_radius
    torque_for_accel.ito("N m")
    torque_for_friction = force_friction * robot_wheel_radius
    torque_for_friction.ito("N m")

    torque_total_per_wheel = (torque_for_accel + torque_for_friction) / robot_num_powered_wheels
    desired_motor_torque = torque_total_per_wheel / EFFICIENCY

    # RPM
    #   RPM = speed / circumference
    rpm = None
    if speed is not None:
        rpm = speed / robot_wheel_radius
        rpm.ito("rpm")

    return desired_motor_torque, rpm

In [ ]:
robot_num_powered_wheels = 4
robot_mass = 10 * units.pounds
robot_wheel_radius = 3.0 * units.inches

table = []
headers = ["Scenario", "RPM", "Torque (Nm)", "Torque (kgf mm)", "Torque (kgf cm)"]

# List of scenarios to account for
scenarios = [
    {
        "index": "A",
        "name": "Max speed on flat",
        "speed": 2.0 * units.mps,
        "accel": 0.0 * units.meter / units.second**2,
        "incline_grade": 0 * units.degrees,
    },
    {
        "index": "B",
        "name": "Accelerating on flat",
        "speed": 2.0 * units.mps,
        "accel": 1.5 * units.meter / units.second**2,
        "incline_grade": 0 * units.degrees,
    },
    {
        "index": "C",
        "name": "Max speed on incline",
        "speed": 1.0 * units.mps,
        "accel": 0.0 * units.meter / units.second**2,
        "incline_grade": 20 * units.degrees,
    },
    {
        "index": "D",
        "name": "Accelerating on incline",
        "speed": 1.0 * units.mps,
        "accel": 0.5 * units.meter / units.second**2,
        "incline_grade": 20 * units.degrees,
    }
]

for scenario in scenarios:
    torque, rpm = calculate_motor_requirements(
        robot_mass,
        robot_num_powered_wheels,
        robot_wheel_radius,
        scenario["speed"],
        scenario["accel"],
        scenario["incline_grade"],
    )

    table.append(
        [
            scenario["index"],
            scenario["name"],
            rpm.to("rpm").magnitude if rpm is not None else None,
            torque.to("N m").magnitude,
            torque.to("kgf mm").magnitude,
            torque.to("kgf cm").magnitude,
        ]
    )

print(tabulate(table, headers=headers, floatfmt=".2f"))